In [48]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

from textacy.extract import ngrams, entities

from collections import Counter
import spacy
nlp = spacy.load('en_core_web_sm')
from tqdm import tqdm

In [142]:
df=pd.read_csv("../Udacity2-Cap/data/fool_tagged.csv")

In [143]:
df.rename(columns={"price":"text"},inplace=True)

In [144]:
df.head()

,title,text,tag
0,Why Williams-Sonoma Stock Popped Today,Shares of were up 11% as of 12:00 p.m. EDT F...,up
1,Why Tailored Brands Stock Dropped 10.5%,Shares of Men's Wearhouse owner stock on Fri...,down
2,Here's Why Mattel Inc Stock Fell Today,Shares of are down 5.9% at 1:50 p.m. on May 3...,down
3,Why Dell Technologies Stock Fell Friday,Shares of information technology and computer ...,down
4,Why Pitney Bowes Stock Dropped 11% on Friday,"It's Friday, and shares of postal solutions pr...",down


# NLTK sentiment

In [145]:
df['scores'] = df['text'].apply(lambda title: sid.polarity_scores(title))
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['comp_score'] = df['compound'].apply(lambda c: 'up' if c >=0 else 'down')

In [146]:
df.tail()

,title,text,tag,scores,compound,comp_score
14262,Why UP Fintech Holding Stock Dropped 16.5%,Shares of recent initial public offering (IPO)...,down,"{'neg': 0.067, 'neu': 0.771, 'pos': 0.162, 'co...",0.9799,up
14263,Why Impinj Shares Fell 13.5% Today,"Shares of started sliding on Friday morning,...",down,"{'neg': 0.033, 'neu': 0.864, 'pos': 0.103, 'co...",0.9570,up
14264,Why Shares of Avaya Are Down Today,Shares of are down more than 13% on Thursday ...,down,"{'neg': 0.051, 'neu': 0.871, 'pos': 0.078, 'co...",0.8442,up
14265,Why Sprint and T-Mobile Shares Jumped Today,"Shares of and have soared today, up by 25% a...",up,"{'neg': 0.011, 'neu': 0.825, 'pos': 0.164, 'co...",0.9925,up
14266,Shares of Warrior Met Coal Surge on Debt and D...,Shares of metallurgical coal miner are up 11...,down,"{'neg': 0.099, 'neu': 0.787, 'pos': 0.114, 'co...",0.3818,up


In [147]:
accuracy_score(df['tag'],df['comp_score'])

0.5929767996074858

# LSVC

In [148]:
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import spacy as spacy
nlp = spacy.load('en_core_web_sm')
def tokenize(text):
    '''this method does the following
    1. normalizing all the words to lower size
    2. removes punctuations
    3. splits the words
    4. removes the stopwords like am,is,have,you,...
    5. lammetizes the words for example running-->run
    6. this does better since it lemmetize for verbs too
    '''
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())    # normalize case and remove punctuation
    tokens = word_tokenize(text)    # tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in nlp.Defaults.stop_words]    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word,pos='v') for word in tokens]    # lemmatize andremove stop words
#     tokens = [PorterStemmer().stem(word) for word in tokens if word not in stop_words]    # lemmatize andremove stop words
    return tokens

In [149]:
X = df['text']  # this time we want to look at the text
y = df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [154]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
def model_Pipeline():
    pipeline=Pipeline([
#         ('tfidf', TfidfVectorizer(max_df=.95,min_df=2,stop_words="english")),
                ('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
#                      ('clf', GradientBoostingClassifier()),
#                      ('clf', RandomForestClassifier())
#                      ('clf', CatBoostClassifier())
#                      ('clf', xgb())
#                         ('clf', LogisticRegression())
#                        ('clf', MultinomialNB())
                    ])
    return pipeline

model=model_Pipeline()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [155]:
accuracy_score(y_test,predictions)

0.8447653429602888

In [104]:
pd.DataFrame(confusion_matrix(y_test,predictions), index=['Pos','Neg'], columns=['Pos','Neg'])

,Pos,Neg
Pos,1855,424
Neg,307,2123


In [120]:
from sklearn.externals import joblib
# joblib.dump(model, 'sentiment/GBClf.pkl')
joblib.dump(model, 'sentiment/LSVCClf.pkl')

['sentiment/LSVCClf.pkl']

In [130]:
model.predict_proba([X_test[0]])

array([[0.34935982, 0.65064018]])

In [131]:
model.predict([X_test[0]])

array(['up'], dtype=object)

In [132]:
from sklearn.externals import joblib

## AlphaSeek

In [2]:
df=pd.read_csv("../scrapy/yahoonews/yahoocrawler/alpha-output2.csv")

In [3]:
df.shape

(74925, 4)

In [6]:
df.dropna(inplace=True)

In [7]:
df.head()

,sym,title1,newstime,desc
1,LMT,Lockheed Martin's Sikorsky wins $1.1B U.S. Nav...,"Today, 9:19 PM",Lockheed Martin's (NYSE:LMT) Sikorsky unit is ...
3,TM,"Toyota slams Trump's latest tariff threat, say...","Today, 6:23 PM","In an unusually strongly worded statement, Toy..."
4,INNT,Innovate Biopharma -3.8% on holders' offering ...,"Today, 6:04 PM",Innovate Biopharmaceuticals (NASDAQ:INNT) is 3...
6,FB,"Facebook sets up Swiss fintech for blockchain,...","Today, 5:53 PM",Facebook (NASDAQ:FB) has established a financi...
7,PDCE,ISS backs two of three Kimmeridge nominees to ...,"Today, 5:51 PM",Activist investor Kimmeridge Energy Management...


In [9]:
df.sym.value_counts(normalize=True).to_frame('count').style.format({'count': '{:,.2%}'.format})

,count
AAPL,0.49%
AMZN,0.47%
TSLA,0.46%
BA,0.43%
GOOG,0.35%
FB,0.34%
GE,0.27%
PCG,0.24%
XOM,0.21%
MSFT,0.18%


In [14]:
df.sym.value_counts().to_frame('count')

,count
AAPL,313
AMZN,300
TSLA,292
BA,273
GOOG,223
...,...
KYKOF,1
HCHDF,1
EERGF,1
DNP,1


#### <font color=green> word count

In [40]:
token_count = Counter()
for i, doc in enumerate(df.desc.tolist(), 1):
    if i % 500 == 0:
        print(i, end=' ', flush=True)
    token_count.update([t for t in doc.split() if t not in nlp.Defaults.stop_words])

500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 17500 18000 18500 19000 19500 20000 20500 21000 21500 22000 22500 23000 23500 24000 24500 25000 25500 26000 26500 27000 27500 28000 28500 29000 29500 30000 30500 31000 31500 32000 32500 33000 33500 34000 34500 35000 35500 36000 36500 37000 37500 38000 38500 39000 39500 40000 40500 41000 41500 42000 42500 43000 43500 44000 44500 45000 45500 46000 46500 47000 47500 48000 48500 49000 49500 50000 50500 51000 51500 52000 52500 53000 53500 54000 54500 55000 55500 56000 56500 57000 57500 58000 58500 59000 59500 60000 60500 61000 61500 62000 62500 63000 63500 

In [36]:
tokens = pd.DataFrame(token_count.most_common(), columns=['token', 'count']).set_index('token')
tokens.head(20)

,count
token,
EPS,33341
beats,18612
Y/Y),18569
GAAP,15471
revenue,13233
misses,11500
estimates,11194
says,10470
company,10197


#### <font color=green> Entity 

In [50]:
entity_count = Counter()
for i, doc in enumerate(df.desc.tolist(), 1):
    if i % 500 == 0:
        print(i, end=' ', flush=True)
    entity_count.update([e.text for e in entities(nlp(doc))])

500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 17500 18000 18500 19000 19500 20000 20500 21000 21500 22000 22500 23000 23500 24000 24500 25000 25500 26000 26500 27000 27500 28000 28500 29000 29500 30000 30500 31000 31500 32000 32500 33000 33500 34000 34500 35000 35500 36000 36500 37000 37500 38000 38500 39000 39500 40000 40500 41000 41500 42000 42500 43000 43500 44000 44500 45000 45500 46000 46500 47000 47500 48000 48500 49000 49500 50000 50500 51000 51500 52000 52500 53000 53500 54000 54500 55000 55500 56000 56500 57000 57500 58000 58500 59000 59500 60000 60500 61000 61500 62000 62500 63000 63500 

In [51]:
entity = pd.DataFrame(entity_count.most_common(), columns=['token', 'count']).set_index('token')
entity.head(20)

,count
token,
NYSE,26027
EPS,8187
Q1,5615
quarterly,5079
GAAP,4993
NASDAQ,4895
U.S.,4614
Q3,3034
EPS Estimate,2922


#### <font color=green> Ngram

In [52]:
ngram_count = Counter()
for i, doc in enumerate(df.desc.tolist(), 1):
    if i % 500 == 0:
        print(i, end=' ', flush=True)
    ngram_count.update([n.text for n in ngrams(nlp(doc), n=2, min_freq=2)])

500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 17500 18000 18500 19000 19500 20000 20500 21000 21500 22000 22500 23000 23500 24000 24500 25000 25500 26000 26500 27000 27500 28000 28500 29000 29500 30000 30500 31000 31500 32000 32500 33000 33500 34000 34500 35000 35500 36000 36500 37000 37500 38000 38500 39000 39500 40000 40500 41000 41500 42000 42500 43000 43500 44000 44500 45000 45500 46000 46500 47000 47500 48000 48500 49000 49500 50000 50500 51000 51500 52000 52500 53000 53500 54000 54500 55000 55500 56000 56500 57000 57500 58000 58500 59000 59500 60000 60500 61000 61500 62000 62500 63000 63500 

In [55]:
ngarm = pd.DataFrame(ngram_count.most_common(), columns=['token', 'count']).set_index('token')
ngarm.head(10)

,count
token,
GAAP EPS,12811
EPS estimates,4310
upward revisions,2276
revenue estimates,1893
Revenue estimates,1889
price target,1010
year ago,958
vs. $,864
seen 0,696


#### <font color=green> Tfidf

In [42]:
tfidf = TfidfVectorizer(stop_words='english')
dtm_tfidf = tfidf.fit_transform(df.desc)
tokens = tfidf.get_feature_names()
dtm_tfidf.shape

(63504, 71080)

In [43]:
pd.DataFrame({'tfidf': dtm_tfidf.sum(axis=0).A1,'token': tokens})

,tfidf,token
0,227.644286,00
1,143.040268,000
2,0.177038,00000001
3,0.167458,00001
4,0.170433,00002
...,...,...
71075,0.342530,β2
71076,0.150438,β7
71077,0.249696,δ9
71078,0.142783,μg


In [47]:
token_freq = (pd.DataFrame({'tfidf': dtm_tfidf.sum(axis=0).A1,
                            'token': tokens})
              .sort_values('tfidf', ascending=False)).set_index('token')
token_freq.head(10)

,tfidf
token,
eps,2713.163925
gaap,2459.132244
revenue,2229.953463
beats,1899.683783
nasdaq,1623.426251
nyse,1531.290325
misses,1386.466083
dividend,1251.232624
share,1199.508952
